In [1283]:
import warnings
warnings.filterwarnings('ignore')

In [1284]:
# Imports
import os
import numpy as np
import pandas as pd 
import csv
import matplotlib.pyplot as plt
from math import sqrt
import datetime

from sklearn import neighbors
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix

from pymongo import MongoClient

In [1285]:
from pymongo import MongoClient
#'database_pipeline'

class MongoDB():
    def __init__(self,dbname):
        self.client = MongoClient()
        self.db = self.client[dbname]

    def get_collection_name(self):
        #Return all collection names
        return self.db.list_collection_names()

    def find(self, collection, what=dict(), _id=False, last=False):
        #by default the function doesn't select the _id field
        #retourne une list de dictionaire last=False
        #retourne un dictionaire si last=True
        if _id == False:
            cursor = self.db[collection].find(what, {'_id': False})
        else:
            cursor = self.db[collection].find(what)

        if last == True:
            cursor = cursor.sort([("Time", -1)]).limit(1)
            return cursor[0]

        return self.cursor_to_dict(cursor)

    def find_all_last(self, what=dict(), _id=False, last=False):
        d = dict()
        for coll in self.get_collection_name():
            if coll == "users":
                continue
            d[coll] = self.find(coll, what, _id, last)
        return d

    def insert_one(self, collection, item):
        try:
            #Insert l'item dans la base de données
            self.db[collection].insert_one(item)
            return 1
        except:
            print("Item non importé")
            return -1
        
    def insert_many(self, collection, liste):
        try:
            #Insert touts les items de la liste dans la base de données
            #En une seule commande
            self.db[collection].insert_many(liste)
            return 1
        except:
            print("Liste d'item non importé")
            return -1

    def cursor_to_dict(self, cursor):
        l = list()
        for i in cursor:
            l.append(i)
        return l

    def get_keys(self, collection):
        #Return all field name of a collection
        map = Code("function() { for (var key in this) { emit(key, null); } }")
        reduce = Code("function(key, stuff) { return null; }")
        result = self.db[collection].map_reduce(map, reduce, "myresults")
        return result.distinct('_id')

#docs = list(self.db[collec_pipe_name].find().sort([('Time', -1)]))


In [1286]:
import importlib
import matplotlib.pyplot as plt

def load_data(path):
    '''
        Ouvre et lis le fichier passer en parmètre, reconnais seulement les types de
        fichier supportés par pandas : CSV, JSON, HTML, Local clipboard, MS Excel,
        HDF5 Format, feather Format, Parquet Format, Msgpack, Stata, SAS, Python Pickle
        Format, SQL, Google Big Query. Retourne les données lu.

        :params:
            path: path of the file

        :type params:
            path: string

        :return: object containing the data loaded in memory, or return -1 if type
                 not recognize.
    '''
    #On récupère le nom de l'extension du fichier
    type = path.split(".")[-1]
    #Selection de la bonne fonction de pandas à utiliser
    func_to_call = 'read_{}'.format(type)

    #Récupération de l'attribut de la fonction pour l'appeler
    try :
        func = getattr(pd, func_to_call)
    except :
        print("Pas de fonction disponible dans pandas pour lire les données")
        return -1

    #Lecture des données
    try :
        return func(path)
    except :
        print("Incorrect path")
        return -1

In [1287]:
def descript_df(dataframe):
    '''
        Permet de faire une description rapide du dataframe de sortie. Retourne une
        description rapide des données.

        :params:
            dataframe: de faire une description rapide du dataframe de sortie

        :type params:
            dataframe: pandas.dataframe

        :return: description rapide des données
        :rtype: string
    '''
    print('Matrice de corrélation : \n')
    corr = dataframe.corr()
    corr_color = plt.matshow(corr, cmap=plt.cm.Reds)

    # Pour chaque colonne, montrer la répartition des valeurs (vérifier les valeurs aberrantes)
    # Kde et Histogramme
    i = 1
    for column in dataframe:
        i += 1
        plt.figure(i, figsize=(15,3))
        plt.subplot(121)
        dataframe[column].plot.kde()
        plt.title('Répartition de ' + column + ' : ')
        plt.subplot(122)
        dataframe[column].hist()
        plt.title('Histogramme de ' + column + ' : ')

    display = corr
    return display

In [1288]:
import random

def bootstrap(pipeline, df, features, target, n):
    '''
        Effectue un boostrap de la pipeline sur les données passées en paramètre.
        
        :params:
            pipeline : pipeline
            df : dataframe
            features : colonnent à utiliser pour la régréssion
            target : colonne à prédire
            n : nombres boostrap à faire

        :return:
            pred : dictionnaire contenant les valeurs bootstrap sous forme de liste pour chaque echantillons.
    '''
    length = len(df)
    keys = range(length)
    pred = {key: list() for key in keys}

    for b in range(n):
        #Random choice
        np.random.seed(b)
        index = np.random.choice(range(length), int(length/0.7))
        index_test = df.index.difference(index)
        train = df.loc[index]
        test = df.loc[index_test]

        #Fit des données
        pipeline.fit(train[features], train[target])

        #Prédiction et score sur la base de test
        for i, p in zip(index_test, pipeline.predict(test[features])):
            pred[i].append(p[0])

    return pred

In [1289]:
from sklearn.model_selection import cross_validate

def compute_regression(pipeline, df, features, target, n):
    '''
        Permet d'obtenir les résultats des indicateurs de performances d'une régression par crossvalidation, 
        et retourne ces derniers.

        :params:
            pipeline : object de type pipeline
            df : tuple des bases d'apprentissage et de test
            features : colonnes à utiliser pour la régréssion
            target : colonne à prédire
            n : nombre de bootstrap pour le calcul des intervalles de confiance
            BDD : booléen, pour True les résultats sont stockés dans la BDD pour
                  False ils ne sont pas sauvegardés
        :return:
            r2 : score R2
            variance : variance gloable expliquée par le modèle
            rmse : root mean squared error
            intervalle_10 : une liste de 10 intervalles de confiances pris dans la list triés
                            des intervalles de confiances.
            intervalle_mean : moyenne de tous les intervalles de confiance
    '''
    
    # Calcul interval de confiance par bootstrap
    pred = bootstrap(pipeline, df, features, target, 200)
    inter_every_x = [2 * np.std(pred[i]) for i in pred.keys()]

    min_inter = min(inter_every_x)
    max_inter = min(inter_every_x)
    med_inter = np.median(inter_every_x)
    
    # Calcul de la variance globale expliqué, de r2 et du RMSE
    # par Cross-Validation
    scoring = {"variance" : "explained_variance",
               "r2" : "r2", 
               "mse" : "neg_mean_squared_error"}
    result = cross_validate(pipeline, df[features], df[target], cv=7, scoring=scoring)
    variance = np.mean(result["test_variance"])
    r2 = np.mean(result["test_r2"])
    rmse = np.mean(np.sqrt(np.absolute(result["test_mse"])))
    
    result = {"R2": r2, "Variance": variance, "RMSE": rmse, "min_inter" : min_inter, "max_inter" : max_inter, "med_inter" : med_inter}

    return result

In [1290]:
def compute_performance(pipeline, modele, df,  features, target, n, BDD=True):
    '''
        Permet d'appeler les bons indicateurs de performances et de récupérer
        les informations utiles selon la pipeline passé en paramètre. Retourne les
        performances associées.

        :params:
            pipeline : object de type pipeline
            modele : type du modèle utilisé : régression/classification)
            features : colonnent à utiliser pour la régréssion
            target : colonne à prédire
            n : nombre de bootstrap pour le calcul des intervalles de confiance
            BDD : booléen, pour True les résultats sont stockés dans la BDD pour
                  False ils ne sont pas sauvegardés

        :type params:
            bool_type_modele: boolean
            base: tuple

        :return: les performances des différents indicateurs et graphiques
    '''
    if (modele == "regression") :
        print("Choix du type d'estimateur : Régression \n")       
        result = compute_regression(pipeline, df, features, target, n)

    elif(modele == "classification"):
        print("Choix du type d'estimateur : Classification \n")
        result = compute_classification(pipeline, df)
    
    else:
        return -1
    
    if BDD == True:
        result['Time'] = datetime.datetime.now()
        result["_id"] = pipeline.name + "." + str(result['Time'])
        mongo = MongoDB("database_pipeline")
        mongo.insert_one(pipeline.name, result)
    
    return result

In [1291]:
def add_metadata_property(obj, name):
    '''
        Permet d'ajouter un atribut à un objet existant
    '''
    setattr(obj, "name", name)

In [1292]:
# Pipeline à utiliser

data = pd.read_csv("./../data/headbrain.csv", skiprows=1, 
                      names=['gender','age_range' , 'head_size', 'brain_weight'])

Y = data_sample.iloc[:,0:3]

# Création de la pipeline
pipe_test = Pipeline([
    ('features', StandardScaler()),
    ('estimator', neighbors.KNeighborsRegressor())   
])

# Ajout d'un nom à la pipeline
add_metadata_property(pipe_test, 'pipe_elias')

In [1293]:
from os import listdir
from os.path import isfile, join

def get_all_pipes_names(path):
    '''
        Liste  tous les fichiers contenus dans le dossier passé en paramètre.
        
        :return: liste contenant les noms des fichiers
    '''
    l = [f for f in listdir(path) if isfile(join(path, f))]
    try :
        l.remove("default.py")
    except:
        pass
    return l

In [1294]:
get_all_pipes_names("./")

['IrisExploration.ipynb',
 'Mi-Avancement Projet.ipynb',
 'pipe_test_1.py',
 'regression.ipynb',
 'pipe_test_2.py',
 'pipe_test_3.py',
 'pipe_test_4.py']

In [1295]:
import importlib.util

def get_pipelines(path, pipe_file):
    '''
        Charge la pipeline dont le nom du fichier est passé en paramètre. Et lui
        rajoute un atribut name contenant son nom.

        :params:
            path: lien du dossier
            pipe_file : nom du fichier de la pipeline.

        :type params:
            pipe_name: str

        :return: object contenant la pipeline
    '''
    spec = importlib.util.spec_from_file_location("module.name", path+pipe_file)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)

    #import de la pipeline
    pipeline = module.pipeline

    # Ajout d'un nom à la pipeline
    add_metadata_property(pipeline, pipe_file[:-3])
    return pipeline

In [1296]:
pipeline = get_pipelines("./", "pipe_test_3.py")

In [1297]:
data_sample = data.sample(frac=0.5,random_state = random.randint(0,1000))
data_sample.reset_index(drop = True, inplace = True)
compute_performance(pipeline, "regression", data_sample, ["age_range", "head_size"], ["brain_weight"], 100)

Choix du type d'estimateur : Régression 



{'R2': 0.46155128599070633,
 'Variance': 0.4832819997784745,
 'RMSE': 81.48474159056632,
 'min_inter': 19.977328415530398,
 'max_inter': 19.977328415530398,
 'med_inter': 63.15216467223158,
 'Time': datetime.datetime(2019, 4, 13, 16, 8, 20, 418591),
 '_id': 'pipe_test_3.2019-04-13 16:08:20.418591'}

In [1298]:
f = [5, 4]
min(f)

4

In [1299]:
len(data)

237